This [Jupyter notebook](https://jupyter.org/)
is meant to be viewed in "slide mode".

If you want to view the slides locally,
you'll need to install [RISE](https://rise.readthedocs.io/).

But it's probably simpler to just visit the
[online version](https://nbviewer.jupyter.org/format/slides/github/mgeier/splines2021/blob/main/presentation.ipynb).

# Rotation Splines for Spatial Audio Scenes

Matthias Geier

https://github.com/mgeier/splines2021

## Context: Audio Scene Description Format (ASDF)

* https://AudioSceneDescriptionFormat.readthedocs.io/
* https://github.com/AudioSceneDescriptionFormat/asdf-rust
* https://github.com/SoundScapeRenderer/ssr/pull/155

* splines for sound source trajectories
* splines for rotation
  * source rotations
  * rotation of groups of sources
  * rotation of local coordinate systems
* splines for volume envelopes

* everything is still *work in progress*!

## Overview

* splines in Euclidean space
  * Bézier splines
    * De Casteljau's algorithm
* three-dimensional rotation
  * unit quaternions
    * Slerp (**s**pherical **l**inear int**erp**olation)
* rotation splines
  * De Casteljau's algorithm (using Slerp)
* Catmull–Rom splines
  * Euclidean space
  * unit quaternions

## Euclidean Space

* where classical geometry happens
* named after Euclid of Alexandria (Εὐκλείδης)
  * book series: "Elements" (Στοιχεῖον)
  * about 300 BCE
* there is exactly one line that passes through two distinct points
* two lines either meet in one point or are parallel

## Splines in Euclidean Space

* known from your favorite vector drawing program
* one-dimensional, two-dimensional, three-dimensional, ...
* https://splines.readthedocs.io/
* https://splines.readthedocs.io/euclidean/

## Parametric Piecewise Cubic Polynomial Curves

\begin{equation*}
\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i
\end{equation*}

* $n$-dimensional
* degree 3 $\to$ four coefficients
* univariate
* uniform vs. non-uniform

## Bézier Splines

* named after Pierre Bézier (1910 – 1999), who worked at Renault
* most common: *cubic* Bézier splines (four control points per segment)

* different ways to calculate points on the curve:
  * evaluate cubic polynomials
  * De Casteljau's algorithm
    * named after Paul de Casteljau (\*1930), who worked at Citroën
    * multiple stages, each only using **l**inear int**erp**olation
      * $\operatorname{Lerp}(\boldsymbol{x}_0, \boldsymbol{x}_1 ;t) = (1 - t) \boldsymbol{x}_0 + t \boldsymbol{x}_1$,
        where $0 \le t \le 1$
    * can be used graphically
    * https://splines.readthedocs.io/euclidean/bezier-de-casteljau.html

## Rotation

* 2D
  * angle
* 3D
  * Euler's rotation theorem (1775) ($\ne$ Euler angles!)
    * axis and angle
    * multiple rotations can be combined into one rotation
  * order of rotations matters!
    * not commutative

## Representing Three-Dimensional Rotation

* azimuth, elevation, roll
  * order matters!
  * global vs local frame of reference
  * gimbal lock
* axis and angle
* rotation matrix
* unit quaternion
* ... and more: homogeneous coordinates, geometric algebra, etc.

## Quaternions

* discovered in 1843 by Sir William Rowan Hamilton
* extension of complex numbers into 3D space
  * ... but four components are needed!
* multiplication is non-commutative

## Representing Quaternions

* algebraic: $w + x\mathbf{i} + y\mathbf{j} + z\mathbf{k}$
  * $\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{ijk} = -1$
  * $\mathbf{ij} = \mathbf{k}$
  * $\mathbf{ji} = -\mathbf{k}$
* scalar and vector: $(w, \vec{v}) = (w, (x, y, z))$
  * "real" and "imaginary"
* 4D vector space (with additional operations)
  * $(w, x, y, z)$ vs. $(x, y, z, w)$
* ... and more: 4x4 real matrix, 2x2 complex matrix, etc.

## Unit Quaternions $\Leftrightarrow$ Rotations

* all quaternions with length $1$: unit hypersphere $S^3$
* $q_i = (\cos \frac{\alpha_i}{2}, \vec{n}_i \sin \frac{\alpha_i}{2})$
  * (normalized) rotation axis $\vec{n}_i$
  * angle $\alpha_i$ (in radians)
* double cover
  * $q$ and $-q$ represent the same rotation!

## Unit Quaternion Operations

* quaternion multiplication: $\; q_1 q_0$
  * rotation $q_0$ followed by rotation $q_1$ (read from right to left)
  * $q_0 q_1 \ne q_1 q_0$ (except if the rotation axis is the same)
* identity: $\; \boldsymbol{1} = (1, (0, 0, 0))$
* inverse: $\; q^{-1}$
  * same rotation axis, negated angle
  * $q q^{-1} = q^{-1} q = \boldsymbol{1}$

## Rotation Splines

* https://splines.readthedocs.io/rotation/
* using unit quaternions
* polynomials cannot be used!
  * cannot use addition with unit quaternions
  * multiplication is non-commutative
* idea: use geometric algorithms involving straight lines
  * e.g. de Casteljau's algorithm

## “Straight Lines” on the Unit Hypersphere

* shortest path between two unit quaternions (i.e. two rotations)
  * great arc (part of a great circle)
  * geodesic
* locally parallel lines will intersect at some point

## Slerp, part 1

* **s**pherical **l**inear int**erp**olation: $\operatorname{Slerp}(q_0, q_1; t)$, where $0 \le t \le 1$
* intermediate step: change angle linearly from $0$ to $\alpha$
  * reminder (axis/angle):
    $q = \left(\cos \frac{\alpha}{2}, \vec{n} \sin \frac{\alpha}{2}\right)$
  * rotation axis $\vec{n}$ stays constant
  * if $\alpha = 0 \to q = \boldsymbol{1} = (1, (0, 0, 0))$
  * $\operatorname{Slerp}(\boldsymbol{1}, q; t) = \left(\cos \frac{\alpha t}{2}, \vec{n} \sin \frac{\alpha t}{2}\right) = q^t$
* $\operatorname{Slerp}(q_0, q_1; t) = \operatorname{Slerp}(\boldsymbol{1}, q_{0,1}; t) \, q_0$
  * $q_{0,1}$: relative rotation from $q_0$ to $q_1$

## Relative Rotation

* $q_{0,1}$: rotation from $q_0$ to $q_1$

\begin{equation*}
q_{0,1} q_0 = q_1
\end{equation*}

* right-multiply both sides with ${q_0}^{-1}$

\begin{equation*}
q_{0,1} q_0 {q_0}^{-1} = q_1 {q_0}^{-1}
\end{equation*}

* $q_0 {q_0}^{-1}$ cancels out

\begin{equation*}
q_{0,1} = q_1 {q_0}^{-1}
\end{equation*}

## Slerp, part 2

* $\operatorname{Slerp}(q_0, q_1; t) = \operatorname{Slerp}(\boldsymbol{1}, q_{0,1}; t) \, q_0$
  * $\operatorname{Slerp}(\boldsymbol{1}, q; t) = q^t$

  * $q_{0,1} = q_1 {q_0}^{-1}$

* $\operatorname{Slerp}(q_0, q_1; t) = \left(q_1 {q_0}^{-1}\right)^t \, q_0$
* https://splines.readthedocs.io/rotation/slerp.html

## De Casteljau's Algorithm with Slerp

* https://splines.readthedocs.io/rotation/de-casteljau.html

## Catmull–Rom Splines

* Euclidean space
  * https://splines.readthedocs.io/euclidean/catmull-rom-properties.html
* unit quaternions
  * https://splines.readthedocs.io/rotation/catmull-rom-uniform.html
  * https://splines.readthedocs.io/rotation/catmull-rom-non-uniform.html

Thanks for your attention!

Questions?

https://github.com/mgeier/splines2021